In [1]:
import networkx as nx 
import matplotlib.pyplot as plt
import json
from utils import *
import pandas as pd
import numpy as np
import pickle as pkl
from collections import defaultdict

import shutil

In [2]:
!pwd

/home/richard/pkgs/deepdrawing


In [3]:
output = "n6e12"

dataset = []
for root, dirs, files in os.walk(f"main_data_folder/data/store/output_{output}/"):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        with open(file_path, "rb") as f:
            g = pkl.load(f)
            dataset.append(g)


In [4]:
len(dataset)

10000

In [5]:
len_list = []
for g in dataset:
    len_list.append(g['len'])


In [6]:
pd.DataFrame(len_list).value_counts()

19    6810
18    2393
17     611
20     186
Name: count, dtype: int64

In [7]:
filter_list = [ i for i in range(len(len_list)) if len_list[i]==19]

# Idea: only transfer graphs with same number of nodes

In [8]:
size = len(filter_list)
part1 = int(80/100 * size)
part2 = int(part1 + 10/100 * size)
part3 = int(part2 + 10/100 * size)
print(part1, part2, part3)


5448 6129 6810


In [9]:
# clear folders
for type in ["train", "test", "valid"]:
    for root, dirs, files in os.walk(f"main_data_folder/data/ortho_{type}_dataset_folder_preprocess/"):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            if os.path.isfile(file_path):
                #print(f"removing file: {file_path}")
                os.remove(file_path)



In [10]:

# for num in filter_list[:part1 - 1]:
for num in filter_list[:800]:
    shutil.copy2(f"main_data_folder/data/store/output_n6e12/output_graphn6e12s150i{num}planar",
                 f"main_data_folder/data/ortho_train_dataset_folder_preprocess/")

# for num in filter_list[part1:part2 - 1]:
for num in filter_list[800:900]:
    shutil.copy2(f"main_data_folder/data/store/output_n6e12/output_graphn6e12s150i{num}planar",
                 f"main_data_folder/data/ortho_test_dataset_folder_preprocess/")

# for num in filter_list[part2:part3 - 1]:
for num in filter_list[900:1000]:
    shutil.copy2(f"main_data_folder/data/store/output_n6e12/output_graphn6e12s150i{num}planar",
                 f"main_data_folder/data/ortho_valid_dataset_folder_preprocess/")

# Draw illustrations of isolated graph types

In [100]:
from data import inv_transform_nodelist
from test import transform_pos


for num in filter_list[:100]:
    with open(f"main_data_folder/data/store/output_n6e12/output_graphn6e12s150i{num}planar" ,"rb") as f:
        g = pkl.load(f)

    
    ori = g['ori']
    y_input = g["pos"]
    y_selected = y_input[g["x_ridx"],:]
    bounding_box = {}
    bounding_box['left'] = 0
    bounding_box['right'] = 300
    bounding_box['top'] = 0
    bounding_box['bottom'] = 300


    pos_ori = inv_transform_nodelist(y_selected,bounding_box,1)
    pos_ori = np.array(pos_ori)
    pos_ori, real_scale_cof = transform_pos(pos_ori,100,100,700,700)

    visualize(pos_ori, ori=ori, name=f"test_{num}", mode=2, test_folder="main_data_folder/drawings/")

# Idea: isolate graphs with same number of nodes AND number of edges

In [50]:
len_edge_list = []
for g in dataset:
    len_edge_list.append(len(g['ori']['linelist']))


In [52]:
filter_list = [ i for i in range(len(len_list)) if len_list[i]==19]

In [55]:
len_edge_filtered_list = [ len_edge_list[i] for i in filter_list ]

In [56]:
pd.DataFrame(len_edge_filtered_list).value_counts()

50    6810
Name: count, dtype: int64

Verdict: its not enough to separate graphs of similar nodes as their order is different

# Idea: Use 

In [58]:
# edge contains (node1, node2), where node = [x,y]
def get_direction(edge):
    node1 = edge[0]
    node2 = edge[1]

    if abs(node1[0] - node2[0]) < 0.1:
        return "vertical"
    elif abs(node1[1] - node2[1]) < 0.1:
        return "horizontal"
    else:
        return "don't know"

In [59]:
def get_bend_points(edge_path):
    bend_points = []
    prev_direction = ""
    for i in range(len(edge_path)):
        # exit condition
        if (i == len(edge_path) - 1):
            break;
        
        # compute current edge direction
        current_path = (edge_path[i], edge_path[i+1])
        direction = get_direction(current_path)
    
        if (i == 0):
            prev_direction = direction
        if (i > 0):
            if ((prev_direction == "vertical" and direction == "horizontal") or 
                (prev_direction == "horizontal" and direction == "vertical")):
                bend_points.append(edge_path[i])
            # set the current direction to be prev for next iteration
            prev_direction = direction

    return bend_points
        


In [60]:
## function to produce modified graph

def create_graph(data):

    G = nx.DiGraph()

    # read in actual nodes first
    node_coordinate = {}
    # first, add the real nodes
    for node_obj in data['nodes']:
        identity = node_obj['id']
        G.add_node(identity)
        node_coordinate[identity] = (node_obj['x'], node_obj['y'], 0)

    # create virtual nodes to split an edge
    node_counter = G.number_of_nodes() -1  # node_id starts from 0
    for edge in data['edges']:
        source = edge['source']
        target = edge['target']
        # full_points_list = [list(node_coordinate[source][0:3])] + edge['path'] + [list(node_coordinate[target][0:3])]
        full_points_list = edge['path']
        # get all bend points only - ignore other points
        bend_points_list = get_bend_points(full_points_list)
        num_points = len(bend_points_list) 
        # connect from real node to first intermediate node
        inter = source
        if num_points == 0:
            G.add_edge(source, target)
            # add reverse edge
            G.add_edge(target, source)
        else:
            for i in range(num_points):
                current_node = bend_points_list[i]
                node_counter = node_counter + 1
                G.add_node(node_counter)
                node_coordinate[node_counter] = (current_node[0], current_node[1], 1)
                G.add_edge(inter, node_counter)
                # add reverse edge
                G.add_edge(node_counter, inter)
                inter = node_counter
            # last edge
            G.add_edge(inter, target)
            # add reverse edge
            G.add_edge(target, inter)
            
    # start of nodelist generation
    nodelist = []

    for node_id in G.nodes:
        # main nodes
        if node_coordinate[node_id][2] == 0:
            node_group = 0
            r = 5
            fill = 'blue'
        # bend nodes
        elif node_coordinate[node_id][2] == 1:
            node_group = 1
            r = 2
            fill = 'orange'
        # alignment nodes
        else:
            node_group = 2
            r = 2
            fill = 'red'

        cx = node_coordinate[node_id][0]
        cy = node_coordinate[node_id][1]
        node = [node_id, node_group, cx, cy, r, fill]
        nodelist.append(node)

    # start of linelist generation
    linelist = G.edges

    return G, nodelist, linelist
        

In [88]:
num = 2
with open(f"main_data_folder/orthogonal/output_n6e12/output_graphn6e12s150i{num}planar.json", "rb") as f:
    data = json.load(f)

G,_,_ = create_graph(data)


In [89]:
import networkx as nx

# Create your planar graph (add nodes and edges)
# G = nx.Graph()
# Add nodes and edges to the graph...

# Check if the graph is planar
is_planar, embedding = nx.check_planarity(G)


In [90]:
len(embedding[0])

4

In [91]:
g

{'ori': {'id': 0,
  'nodelist': [[0, 0, 80.1444444444, 100.35, 5, 'blue'],
   [1, 0, 80.1444444444, 60.15, 5, 'blue'],
   [2, 0, 80.1444444444, 80.25, 5, 'blue'],
   [3, 0, 40.05, 60.1055555556, 5, 'blue'],
   [4, 0, 120.2388888889, 60.1055555556, 5, 'blue'],
   [5, 0, 80.1444444444, 40.05, 5, 'blue'],
   [6, 1, 20.0, 100.3444444444, 2, 'orange'],
   [7, 1, 20.0, 60.1, 2, 'orange'],
   [8, 1, 40.0444444444, 80.2555555556, 2, 'orange'],
   [9, 1, 120.2444444444, 80.2555555556, 2, 'orange'],
   [10, 1, 120.2444444444, 20.0, 2, 'orange'],
   [11, 1, 40.0444444444, 20.0, 2, 'orange'],
   [12, 1, 140.2888888889, 60.1, 2, 'orange'],
   [13, 1, 140.2888888889, 100.3444444444, 2, 'orange'],
   [14, 1, 100.1888888889, 60.0777777778, 2, 'orange'],
   [15, 1, 100.1888888889, 40.0777777778, 2, 'orange'],
   [16, 1, 60.1, 60.0777777778, 2, 'orange'],
   [17, 1, 60.1, 40.0777777778, 2, 'orange']],
  'linelist': [(0, 6),
   (0, 2),
   (0, 13),
   (1, 2),
   (1, 3),
   (1, 5),
   (1, 4),
   (2, 1),
  